The validation scheme is based on [seq2seq-rnn-with-gru](https://www.kaggle.com/brandenkmurray/seq2seq-rnn-with-gru/output), and cleaned data is from [data-without-drift](https://www.kaggle.com/cdeotte/data-without-drift) and Kalman filter is from [https://www.kaggle.com/teejmahal20/single-model-lgbm-kalman-filter](single-model-lgbm-kalman-filter) and the added feature is from [wavenet-with-1-more-feature](wavenet-with-1-more-feature). I also used ragnar's data in this version [clean-kalman](https://www.kaggle.com/ragnar123/clean-kalman). The Wavenet is based on [https://github.com/philipperemy/keras-tcn](https://github.com/philipperemy/keras-tcn), [https://github.com/peustr/wavenet](https://github.com/peustr/wavenet) and [https://github.com/basveeling/wavenet](https://github.com/basveeling/wavenet) and also [https://www.kaggle.com/wimwim/wavenet-lstm](https://www.kaggle.com/wimwim/wavenet-lstm). If any refrence is not mentioned it was not intentional, please add them in comments.

Previous versions were mainly based on [https://www.kaggle.com/wimwim/wavenet-lstm](https://www.kaggle.com/wimwim/wavenet-lstm)  

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Apr 20 07:07:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install --no-warn-conflicts -q tensorflow-addons

In [0]:
from tensorflow.keras.layers import (TimeDistributed, Dropout, BatchNormalization, Flatten, Convolution1D, Activation, Input, Dense, LSTM, Lambda, Bidirectional,
                                     Add, AveragePooling1D, Multiply, GRU, GRUCell, LSTMCell, SimpleRNNCell, SimpleRNN, TimeDistributed, RNN, SpatialDropout1D,
                                     RepeatVector, Conv1D, MaxPooling1D, Concatenate, GlobalAveragePooling1D, UpSampling1D, Layer)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, ReduceLROnPlateau, LearningRateScheduler, CSVLogger
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy, mean_squared_error
# from tensorflow.keras.experimental import export_saved_model, load_from_saved_model
from tensorflow.keras.initializers import Ones, Zeros
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras import losses, models, optimizers
from tensorflow.keras import backend as K
from tensorflow.python.ops import array_ops
import tensorflow as tf
from typing import List, NoReturn, Union, Tuple, Optional, Text, Generic, Callable, Dict
from sklearn.metrics import f1_score, cohen_kappa_score, mean_squared_error
from logging import getLogger, Formatter, StreamHandler, FileHandler, INFO, DEBUG, NOTSET
from sklearn.model_selection import KFold, GroupKFold
from tqdm import tqdm_notebook as tqdm
from contextlib import contextmanager
from joblib import Parallel, delayed
from IPython.display import display
from sklearn import preprocessing
from sklearn.utils import class_weight
import tensorflow_addons as tfa
import scipy.stats as stats
import random as rn
import pandas as pd
import numpy as np
import scipy as sp
import itertools
import warnings
import time
import pywt
import os
import gc

from tensorflow.keras.metrics import Precision, Recall
# from tensorflow_addons.metrics import F1Score

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 500)
%matplotlib inline

In [0]:
Kaggle = False
Colab = True

In [5]:
import os, sys
from pathlib import Path

if Colab:
    from google.colab import drive
    drive.mount('/content/drive')

    path = "/content/drive/My Drive"

    os.chdir(path)
    os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# sys.path.append('ion_switch/keras-one-cycle')
# # os.listdir(patholr)
# from clr import OneCycleLR

In [0]:
if Kaggle:
    PATH = '/kaggle/input/'
    outdir = '.'
# PATH = '/Users/helen/Desktop/Data/'
else:
    PATH = 'ion_switch/'
    outdir = Path(PATH+'res')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    outdir = Path(PATH+'res/wavenet-lr-res')
    if not os.path.exists(outdir):
        os.mkdir(outdir)

In [0]:
EPOCHS=300
NNBATCHSIZE=20
BATCHSIZE = 4000
SEED = 321
SELECT = True
SPLITS = 5
LR = 0.001
Gamma = 1.0 #0.99994
BETA = 0.99996
fe_config = [
    (True, BATCHSIZE),
]

COMPETITION = 'ION-Switching'
logger = getLogger(COMPETITION)
LOGFORMAT = '%(asctime)s %(levelname)s %(message)s'
MODELNAME = 'WaveNet-NormForward-LR-FOCALLOSS'

TRAINEDMODEL = os.path.join(outdir, 'wavenet-es-v1/wavenet_es_f0_checkpoint.h5')

VERSION = '{}_len{}_lr{}'.format(MODELNAME, BATCHSIZE, LR)
outdir = os.path.join(outdir, VERSION)
if not os.path.exists(outdir):
    os.mkdir(outdir)

from datetime import datetime
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y-%H")

outdir = os.path.join(outdir, timestampStr)
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [0]:

@contextmanager
def timer(name : Text):
    t0 = time.time()
    yield
    logger.info(f'[{name}] done in {time.time() - t0:.0f} s')

In [0]:

def init_logger():

    handler = StreamHandler()
    handler.setLevel(INFO)
    handler.setFormatter(Formatter(LOGFORMAT))
    fh_handler = FileHandler(os.path.join(outdir,'{}-len{}-lr{}-{}.log'.format(MODELNAME,BATCHSIZE,LR,timestampStr)))
    fh_handler.setFormatter(Formatter(LOGFORMAT))
    logger.setLevel(INFO)
    logger.addHandler(handler)
    logger.addHandler(fh_handler)
    

In [0]:

def seed_everything(seed : int) -> NoReturn :
    
    rn.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    # os.environ['TF_CUDNN_DETERMINISTIC'] = str(seed) 

seed_everything(SEED)

In [0]:
class CyclicLR(tf.keras.callbacks.Callback):

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2. ** (x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** (x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(
                self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        K.set_value(self.model.optimizer.lr, self.clr())
        # print("learning rate- self.model.optimizer.lr: ", self.model.optimizer.lr)

    # def on_epoch_end(self, epoch, logs=None):

    #     logs = logs or {}
    #     self.trn_iterations += 1
    #     self.clr_iterations += 1

    #     K.set_value(self.model.optimizer.lr, self.clr())
    #     logger.info(f'epoch:{epoch:03d},'+str(K.eval(self.model.optimizer.lr)))

In [0]:
import matplotlib.pyplot as plt

# 写一个LossHistory类，保存训练集的loss和acc
# 当然我也可以完全不这么做，可以直接使用model.fit()方法返回的 history对象去做
'''Callback有6个常用的方法，这里实现其中的四个
    def on_epoch_begin(self, epoch, logs=None):
    def on_epoch_end(self, epoch, logs=None):
    def on_batch_begin(self, batch, logs=None):
    def on_batch_end(self, batch, logs=None):
    def on_train_begin(self, logs=None):
    def on_train_end(self, logs=None):
'''
class LossHistory(Callback):  # 继承自Callback类
 
    '''
    在模型开始的时候定义四个属性，每一个属性都是字典类型，存储相对应的值和epoch
    '''
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}
 
    # 在每一个batch结束后记录相应的值
    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('accuracy'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_accuracy'))
    
    # 在每一个epoch之后记录相应的值
    def on_epoch_end(self, epoch, logs={}):
        trloss, tracc, vloss, vacc = logs.get('loss'), logs.get('accuracy'), logs.get('val_loss'), logs.get('val_accuracy')
        self.losses['epoch'].append(trloss)
        self.accuracy['epoch'].append(tracc)
        self.val_loss['epoch'].append(vloss)
        self.val_acc['epoch'].append(vacc)
        logger.info("epoch:{:03d}, train_loss:{:1.5f}, train_acc:{:1.5f}, val_loss:{:1.5f}, val_acc:{:1.5f}".format(epoch, 
                                                                                                                trloss, tracc, vloss, vacc))
 
    def loss_plot(self, loss_type, pngname):
        '''
        loss_type：指的是 'epoch'或者是'batch'，分别表示是一个batch之后记录还是一个epoch之后记录
        '''
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.savefig(pngname)
        plt.show()

In [0]:

def read_data(base : os.path.abspath) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    
    train = pd.read_csv(PATH+'clean-kalman/train_clean_kalman.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
    test  = pd.read_csv(PATH+'clean-kalman/test_clean_kalman.csv', dtype={'time': np.float32, 'signal': np.float32})
    sub  = pd.read_csv(PATH+'liverpool-ion-switching/sample_submission.csv', dtype={'time': np.float32})
    
    return train, test, sub


In [0]:

def batching(df : pd.DataFrame,
             batch_size : int) -> pd.DataFrame :
    
    df['group'] = df.groupby(df.index//batch_size, sort=False)['signal'].agg(['ngroup']).values
    df['group'] = df['group'].astype(np.uint16)
        
    return df


In [0]:

def reduce_mem_usage(df: pd.DataFrame,
                     verbose: bool = True) -> pd.DataFrame:
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':

                if (c_min > np.iinfo(np.int32).min
                      and c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min
                      and c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min
                        and c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min
                      and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    reduction = (start_mem - end_mem) / start_mem

    msg = f'Mem. usage decreased to {end_mem:5.2f} MB ({reduction * 100:.1f} % reduction)'
    if verbose:
        print(msg)

    return df


In [0]:

def lag_with_pct_change(df : pd.DataFrame,
                        shift_sizes : Optional[List]=[1, 2],
                        add_pct_change : Optional[bool]=False,
                        add_pct_change_lag : Optional[bool]=False,
                        add_diff : Optional[bool]=False) -> pd.DataFrame:
    
    for shift_size in shift_sizes:    
        df['signal_shift_pos_'+str(shift_size)] = df.groupby('group')['signal'].shift(shift_size).fillna(0)
        df['signal_shift_neg_'+str(shift_size)] = df.groupby('group')['signal'].shift(-1*shift_size).fillna(0)

    if add_pct_change:
        df['pct_change'] = df['signal'].pct_change()
        if add_pct_change_lag:
            for shift_size in shift_sizes:    
                df['pct_change_shift_pos_'+str(shift_size)] = df.groupby('group')['pct_change'].shift(shift_size).fillna(0)
                df['pct_change_shift_neg_'+str(shift_size)] = df.groupby('group')['pct_change'].shift(-1*shift_size).fillna(0)
    if add_diff:
        for c in [c1 for c1 in df.columns if c1 not in ['time', 'signal', 'open_channels', 'group', 'index']]:
            df[c+'_msignal'] = df[c] - df['signal']
    return df


In [0]:

def run_feat_enginnering(df : pd.DataFrame,
                         create_all_data_feats : bool,
                         batch_size : int) -> pd.DataFrame:
    
    df = batching(df, batch_size=batch_size)
    if create_all_data_feats:
        df = lag_with_pct_change(df, [1, 2, 3],  add_pct_change=False, add_pct_change_lag=False, add_diff=True)
    df['signal_2'] = df['signal'] ** 2
    return df


In [0]:
def feature_selection(df : pd.DataFrame,
                      df_test : pd.DataFrame) -> Tuple[pd.DataFrame , pd.DataFrame, List]:
    use_cols = [col for col in df.columns if col not in ['index','group', 'open_channels', 'time']]
    print(use_cols)
    df = df.replace([np.inf, -np.inf], np.nan)
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    for col in use_cols:
        col_mean = pd.concat([df[col], df_test[col]], axis=0).mean()
        df[col] = df[col].fillna(col_mean)
        df_test[col] = df_test[col].fillna(col_mean)
   
    gc.collect()
    return df, df_test, use_cols


In [0]:

def augment(X: np.array, y:np.array) -> Tuple[np.array, np.array]:
    
    X = np.vstack((X, np.flip(X, axis=1)))
    y = np.vstack((y, np.flip(y, axis=1)))
    
    return X, y

In [0]:
# Add ops to save and restore all the variables.
# saver = tf.train.Saver()

In [0]:
# # %% [code] {"ExecuteTime":{"end_time":"2020-04-03T23:24:41.652529Z","start_time":"2020-04-03T23:24:41.645025Z"}}
# class EarlyStopping:
#     def __init__(self, patience=5, delta=0, checkpoint_path='checkpoint.pt', is_maximize=True):
#         self.patience, self.delta, self.checkpoint_path = patience, delta, checkpoint_path
#         self.counter, self.best_score = 0, None
#         self.is_maximize = is_maximize

#     def load_best_weights(self, sess):
#         saver.restore(sess, self.checkpoint_path)

#     def __call__(self, score, sess):
#         if self.best_score is None or \
#         (score > self.best_score + self.delta if self.is_maximize else score < self.best_score - self.delta):
#             saver.save(sess, self.checkpoint_path)
#             self.best_score, self.counter = score, 0
#         else:
#             self.counter += 1
#             if self.counter >= self.patience:
#                 return True
#         return False

In [0]:

def run_cv_model_by_batch(train : pd.DataFrame,
                          test : pd.DataFrame,
                          splits : int,
                          batch_col : Text,
                          feats : List,
                          sample_submission: pd.DataFrame,
                          nn_epochs : int,
                          nn_batch_size : int) -> NoReturn:
    seed_everything(SEED)
    K.clear_session()
    if not os.path.exists(outdir):
      os.mkdir(outdir)
    print(outdir)
    config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
    tf.compat.v1.keras.backend.set_session(sess)
    oof_ = np.zeros((len(train), 11))
    preds_ = np.zeros((len(test), 11))
    target = ['open_channels']
    group = train['group']
    kf = GroupKFold(n_splits=5)
    splits = [x for x in kf.split(train, train[target], group)]

    new_splits = []
    for sp in splits:
        new_split = []
        new_split.append(np.unique(group[sp[0]]))
        new_split.append(np.unique(group[sp[1]]))
        new_split.append(sp[1])    
        new_splits.append(new_split)

    # Calculate the weights for each class so that we can balance the data
    weights_ = class_weight.compute_class_weight('balanced',
                                                np.unique(train.open_channels),
                                                train.open_channels)
    print("weights_:", weights_)

    nums_ = train.open_channels.value_counts(sort=False)
    print("nums_:", nums_)
    beta = BETA
    effective_num = 1.0 - np.power(beta, nums_)
    weights = (1.0 - beta) / np.array(effective_num)
    cb_weights_ = weights / np.sum(weights) * 11
    print("cb_weights_:", cb_weights_)   
        
    tr = pd.concat([pd.get_dummies(train.open_channels), train[['group']]], axis=1)

    tr.columns = ['target_'+str(i) for i in range(11)] + ['group']
    # print(tr.head())
    target_cols = ['target_'+str(i) for i in range(11)]
    train_tr = np.array(list(tr.groupby('group').apply(lambda x: x[target_cols].values))).astype(np.float32)
#     print(np.shape(train_tr))
    train = np.array(list(train.groupby('group').apply(lambda x: x[feats].values)))
#     print(np.shape(train))
    test = np.array(list(test.groupby('group').apply(lambda x: x[feats].values)))
#     print(np.shape(test))

    for n_fold, (tr_idx, val_idx, val_orig_idx) in enumerate(new_splits[0:], start=0):
        sub_dir = os.path.join(outdir,"{}_fold".format(n_fold))
        if n_fold == 0:
            continue
        if not os.path.exists(sub_dir):
            os.mkdir(sub_dir)
        
        train_x, train_y = train[tr_idx], train_tr[tr_idx]
        valid_x, valid_y = train[val_idx], train_tr[val_idx]
        
        # if n_fold < 2:
        train_x, train_y = augment(train_x, train_y)

        gc.collect()
        shape_ = (None, train_x.shape[2])
        model = ClassifierD2(shape_, cb_weights_)
        print("model initilization done!")
        # cb_lr_schedule = LearningRateScheduler(lr_schedule)
        cb_clr = CyclicLR(base_lr=1e-7, max_lr = LR, step_size= int(1.0*(train.shape[0])/(nn_batch_size*4)) , 
                          mode='exp_range', gamma=Gamma, scale_fn=None, scale_mode='cycle')
        lr_scheduler = LRSchedulerPerStep(256, 4000) 
        cb_prg = tfa.callbacks.TQDMProgressBar(leave_epoch_progress=False,leave_overall_progress=False, 
                                               show_epoch_progress=False,show_overall_progress=True)
        # cb_csv_logger= CSVLogger(os.path.join(sub_dir,'res.csv'))
        # cb_history = LossHistory()  # 这里是使用自定义的Callback回调函数，当然本身fit函数也会返回一个history可供使用
        
        if Kaggle:
            save_checkpoint_path = os.path.join(sub_dir,'checkpoint-acc-modelonly-{}.h5'.format(n_fold))
            save_finalmodel_path = os.path.join(sub_dir,'fmodel-modelonly-{}.h5'.format(n_fold))
            save_bestf1macro_path = os.path.join(sub_dir,'checkpoint-modelonly-{}.h5'.format(n_fold)) 
            cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_checkpoint_path,
                                                 monitor='val_get_f1',
                                                 mode = 'max',            
                                                 save_weights_only=True,
                                                 save_best_only=True,           
                                                 verbose=1)
            if os.path.exists(save_bestf1macro_path):
                      model.load_weights(save_bestf1macro_path)

            history = model.fit(train_x,train_y,
                              epochs=nn_epochs,
                              callbacks=[cb_prg, lr_scheduler, cp_callback,
                                          MacroF1ES(model, valid_x, valid_y, patience=30, delta=0, 
                                                    checkpoint_path=save_bestf1macro_path)],
                              batch_size=nn_batch_size,
                              verbose=1,
                              validation_data=(valid_x,valid_y))
        else:
            save_checkpoint_path = os.path.join(sub_dir,'checkpoint-acc-modelonly-{}.h5'.format(n_fold))
            save_finalmodel_path = os.path.join(sub_dir,'fmodel-modelonly-{}.h5'.format(n_fold))
            save_bestf1macro_path = os.path.join(sub_dir,'checkpoint-modelonly-{}.h5'.format(n_fold)) 
            # if os.path.exists(save_bestf1macro_path):
            #       model = tf.keras.models.load_model(save_bestf1macro_path, custom_objects={'Activation': tf.keras.layers.Activation, 
            #                                                                               'get_f1':get_f1, 'multi_category_focal_loss2_fixed':multi_category_focal_loss2})
            #       print("load model!!!")
            history = model.fit(train_x,train_y,
                  epochs=nn_epochs,
                  callbacks=[cb_prg, lr_scheduler,
                              MacroF1ES(model, valid_x, valid_y, patience=30, delta=0, 
                                        checkpoint_path=save_bestf1macro_path)],
                  batch_size=nn_batch_size,
                  verbose=1,
                  validation_data=(valid_x,valid_y))

        pd.DataFrame(history.history).to_csv(os.path.join(sub_dir,'{}-len{}-lr{}-{}-log-{}.csv'.format(MODELNAME,BATCHSIZE,LR,timestampStr, n_fold)), float_format='%.4f')
        # print('\nhistory dict:', history.history)
        # cb_history.loss_plot('epoch',os.path.join(outdir,'{}_f{}.png'.format(VERSION, n_fold)))
        model.save_weights(save_finalmodel_path)
#         model = ClassifierD2(shape_, cb_weights_)
#         if Colab:
#             model = tf.keras.models.load_model(save_bestf1macro_path, custom_objects={'Activation': tf.keras.layers.Activation, 
#                                                                                      'get_f1':get_f1, 'multi_category_focal_loss2_fixed':multi_category_focal_loss2})
            
#             # model = tf.keras.models.load_model(save_checkpoint_path, custom_objects={'Activation': tf.keras.layers.Activation, 
#             #                                                                   'get_f1':get_f1, 'loss':categorical_focal_loss})
# #             model = tf.keras.models.load_model(save_checkpoint_path, custom_objects={'Activation': tf.keras.layers.Activation, 
# #                                                                             'get_f1':get_f1, 'cb_focal_loss_fixed':cb_focal_loss})
#         elif Kaggle:
#             model.load_weights(save_checkpoint_path)
        model.load_weights(save_bestf1macro_path)
        preds_f = model.predict(valid_x)
        f1_score_ = f1_score(np.argmax(valid_y, axis=2).reshape(-1),  
                             np.argmax(preds_f, axis=2).reshape(-1), average = 'macro')
        logger.info(f'Training fold {n_fold + 1} completed. macro f1 score : {f1_score_ :1.5f}')
        preds_f = preds_f.reshape(-1, preds_f.shape[-1])
        oof_[val_orig_idx,:] += preds_f
        te_preds = model.predict(test)
        te_preds = te_preds.reshape(-1, te_preds.shape[-1])           
        preds_ += te_preds / SPLITS

    f1_score_ =f1_score(np.argmax(train_tr, axis=2).reshape(-1),  np.argmax(oof_, axis=1), average = 'macro')
    logger.info(f'Training completed. oof macro f1 score : {f1_score_:1.5f}')
    sample_submission['open_channels'] = np.argmax(preds_, axis=1).astype(int)
    sample_submission.to_csv(os.path.join(outdir,'{}_pred.csv'.format(VERSION)), index=False, float_format='%.4f')
    display(sample_submission.head())
    # np.save(os.path.join(outdir,'oof.npy'), oof_)
    # np.save(os.path.join(outdir,'preds.npy'), preds_)

    return 


In [0]:
def lr_schedule(epoch):
    if epoch < 40:
        lr = LR
    elif epoch < 50:
        lr = LR / 3
    elif epoch < 60:
        lr = LR / 6
    elif epoch < 75:
        lr = LR / 9
    elif epoch < 85:
        lr = LR / 12
    elif epoch < 100:
        lr = LR / 15
    else:
        lr = LR / 50
    return lr

In [0]:
class Mish(tf.keras.layers.Layer):

    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)
        self.supports_masking = True

    def call(self, inputs):
        return inputs * K.tanh(K.softplus(inputs))

    def get_config(self):
        base_config = super(Mish, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape
def mish(x):
	return tf.keras.layers.Lambda(lambda x: x*K.tanh(K.softplus(x)))(x)
 
from tensorflow.keras.utils import get_custom_objects
# from tensorflow.keras.layers import Activation
get_custom_objects().update({'mish': Activation(mish)})

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras import constraints

class Attention(Layer):
    """Multi-headed attention layer."""
    
    def __init__(self, hidden_size, 
                 num_heads = 8, 
                 attention_dropout=.1,
                 trainable=True,
                 name='Attention'):
        
        if hidden_size % num_heads != 0:
            raise ValueError("Hidden size must be evenly divisible by the number of heads.")
            
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.trainable = trainable
        self.attention_dropout = attention_dropout
        self.dense = tf.keras.layers.Dense(self.hidden_size, use_bias=False)
        super(Attention, self).__init__(name=name)

    def split_heads(self, x):
        """Split x into different heads, and transpose the resulting value.
        The tensor is transposed to insure the inner dimensions hold the correct
        values during the matrix multiplication.
        Args:
          x: A tensor with shape [batch_size, length, hidden_size]
        Returns:
          A tensor with shape [batch_size, num_heads, length, hidden_size/num_heads]
        """
        with tf.name_scope("split_heads"):
            batch_size = tf.shape(x)[0]
            length = tf.shape(x)[1]

            # Calculate depth of last dimension after it has been split.
            depth = (self.hidden_size // self.num_heads)

            # Split the last dimension
            x = tf.reshape(x, [batch_size, length, self.num_heads, depth])

            # Transpose the result
            return tf.transpose(x, [0, 2, 1, 3])
    
    def combine_heads(self, x):
        """Combine tensor that has been split.
        Args:
          x: A tensor [batch_size, num_heads, length, hidden_size/num_heads]
        Returns:
          A tensor with shape [batch_size, length, hidden_size]
        """
        with tf.name_scope("combine_heads"):
            batch_size = tf.shape(x)[0]
            length = tf.shape(x)[2]
            x = tf.transpose(x, [0, 2, 1, 3])  # --> [batch, length, num_heads, depth]
            return tf.reshape(x, [batch_size, length, self.hidden_size])        

    def call(self, inputs):
        """Apply attention mechanism to inputs.
        Args:
          inputs: a tensor with shape [batch_size, length_x, hidden_size]
        Returns:
          Attention layer output with shape [batch_size, length_x, hidden_size]
        """
        # Google developper use tf.layer.Dense to linearly project the queries, keys, and values.
        q = self.dense(inputs)
        k = self.dense(inputs)
        v = self.dense(inputs)

        q = self.split_heads(q)
        k = self.split_heads(k)
        v = self.split_heads(v)
        
        # Scale q to prevent the dot product between q and k from growing too large.
        depth = (self.hidden_size // self.num_heads)
        q *= depth ** -0.5
        
        logits = tf.matmul(q, k, transpose_b=True)
        # logits += self.bias
        weights = tf.nn.softmax(logits, name="attention_weights")
        
        if self.trainable:
            weights = tf.nn.dropout(weights, 1.0 - self.attention_dropout)
        
        attention_output = tf.matmul(weights, v)
        attention_output = self.combine_heads(attention_output)
        attention_output = self.dense(attention_output)
        return attention_output
        
    def compute_output_shape(self, input_shape):
        return tf.TensorShape(input_shape)

In [0]:
def categorical_focal_loss(gamma=2.0, alpha=0.25):
    """
    Implementation of Focal Loss from the paper in multiclass classification
    Formula:
        loss = -alpha*((1-p)^gamma)*log(p)
    Parameters:
        alpha -- the same as wighting factor in balanced cross entropy
        gamma -- focusing parameter for modulating factor (1-p)
    Default value:
        gamma -- 2.0 as mentioned in the paper
        alpha -- 0.25 as mentioned in the paper
    """
    epsilon = 1.e-7
    gamma = float(gamma)
    alpha = tf.constant(alpha, dtype=tf.float32)
    def loss(y_true, y_pred):
        # Define epsilon so that the backpropagation will not result in NaN
        # for 0 divisor case
        # epsilon = K.epsilon()
        # Add the epsilon to prediction value
        #y_pred = y_pred + epsilon
        # Clip the prediction value
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0-epsilon)
        # Calculate cross entropy
        cross_entropy = -tf.multiply(y_true,tf.math.log(y_pred))
        # Calculate weight that consists of  modulating factor and weighting factor
        weight = tf.multiply(y_true , tf.pow((tf.subtract(1.,y_pred)), gamma))
        weight = tf.multiply(alpha, weight)
        # Calculate focal loss
        wloss = tf.multiply(weight , cross_entropy)
        # Sum the losses in mini_batch
        wloss = tf.reduce_mean(wloss)
        return wloss

    return loss

In [0]:
# focal loss with one-hot labels, multiclass
def focal_loss(classes_num, gamma=2., alpha=.25, e=3e-4):
    # classes_num contains sample number of each classes
    def focal_loss_fixed(target_tensor, prediction_tensor):
        '''
        prediction_tensor is the output tensor with shape [None, 100], where 100 is the number of classes
        target_tensor is the label tensor, same shape as predcition_tensor
        '''

        #1# get focal loss with no balanced weight which presented in paper function (4)
        zeros = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)
        one_minus_p = array_ops.where(tf.greater(target_tensor,zeros), target_tensor - prediction_tensor, zeros)
        FT = -1 * (one_minus_p ** gamma) * tf.math.log(tf.clip_by_value(prediction_tensor, 1e-8, 1.0))

        #2# get balanced weight alpha
        classes_weight = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)

        total_num = float(sum(classes_num))
        classes_w_t1 = [ total_num / ff for ff in classes_num ]
        sum_ = sum(classes_w_t1)
        classes_w_t2 = [ ff/sum_ for ff in classes_w_t1 ]   #scale
        classes_w_tensor = tf.convert_to_tensor(classes_w_t2, dtype=prediction_tensor.dtype)
        classes_weight += classes_w_tensor

        alpha = array_ops.where(tf.greater(target_tensor, zeros), classes_weight, zeros)

        #3# get balanced focal loss
        balanced_fl = alpha * FT
        balanced_fl = tf.reduce_mean(balanced_fl)

        #4# add other op to prevent overfit
        # reference : https://spaces.ac.cn/archives/4493
        nb_classes = len(classes_num)
        fianal_loss = (1-e) * balanced_fl + e * K.categorical_crossentropy(K.ones_like(prediction_tensor)/nb_classes, prediction_tensor)

        return fianal_loss
    return focal_loss_fixed

In [0]:
# focal loss with one-hot labels, multiclass
def cb_focal_loss(classes_weights, gamma=2., alpha=.25, w=3e-4):
    # classes_weights contains weights of each classes
    def cb_focal_loss_fixed(target_tensor, prediction_tensor):
        '''
        prediction_tensor is the output tensor with shape [None, 100], where 100 is the number of classes
        target_tensor is the label tensor, same shape as predcition_tensor
        '''

        #1# get focal loss with no balanced weight which presented in paper function (4)
        zeros = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)
        one_minus_p = array_ops.where(tf.greater(target_tensor,zeros), target_tensor - prediction_tensor, zeros)
        FT = -1 * (one_minus_p ** gamma) * tf.math.log(tf.clip_by_value(prediction_tensor, 1e-8, 1.0))

        #2# get balanced weight alpha
        classes_weight = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)
        classes_w_tensor = tf.convert_to_tensor(classes_weights, dtype=prediction_tensor.dtype)
        classes_weight += classes_w_tensor

        alpha = array_ops.where(tf.greater(target_tensor, zeros), classes_weight, zeros)

        #3# get balanced focal loss
        balanced_fl = alpha * FT
        balanced_fl = tf.reduce_mean(balanced_fl)

        #4# add other op to prevent overfit
        # reference : https://spaces.ac.cn/archives/4493
        nb_classes = len(classes_weights)
        fianal_loss = (1-w) * balanced_fl + w * K.categorical_crossentropy(K.ones_like(prediction_tensor)/nb_classes, prediction_tensor)

        return fianal_loss
    return cb_focal_loss_fixed

In [0]:
def multi_category_focal_loss2(gamma=2., alpha=.25):
    """
    focal loss for multi category of multi label problem
    适用于多分类或多标签问题的focal loss
    alpha控制真值y_true为1/0时的权重
        1的权重为alpha, 0的权重为1-alpha
    当你的模型欠拟合，学习存在困难时，可以尝试适用本函数作为loss
    当模型过于激进(无论何时总是倾向于预测出1),尝试将alpha调小
    当模型过于惰性(无论何时总是倾向于预测出0,或是某一个固定的常数,说明没有学到有效特征)
        尝试将alpha调大,鼓励模型进行预测出1。
    Usage:
     model.compile(loss=[multi_category_focal_loss2(alpha=0.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    epsilon = 1.e-7
    gamma = float(gamma)
    alpha = tf.constant(alpha, dtype=tf.float32)

    def multi_category_focal_loss2_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    
        alpha_t = y_true*alpha + (tf.ones_like(y_true)-y_true)*(1-alpha)
        y_t = tf.multiply(y_true, y_pred) + tf.multiply(1-y_true, 1-y_pred)
        ce = -tf.math.log(y_t)
        weight = tf.pow(tf.subtract(1., y_t), gamma)
        fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
        loss = tf.reduce_mean(fl)
        return loss
    return multi_category_focal_loss2_fixed

In [0]:
def f1v2(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
def f1macro(y_true, y_pred):
    b,l,c = np.shape(y_true)
    y_pred = K.round(y_pred)
    tp = K.sum(K.sum(K.cast(y_true*y_pred, 'float'), axis=2),axis=1)
    tn = K.sum(K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=2),axis=1)
    fp = K.sum(K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=2),axis=1)
    fn = K.sum(K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=2),axis=1)
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [0]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [0]:
def fbeta_score_macro(y_true, y_pred, beta=1, threshold=0.5):

    y_true = K.cast(y_true, 'float')
    y_pred = K.cast(K.greater(K.cast(y_pred, 'float'), threshold), 'float')

    tp = K.sum(y_true * y_pred, axis=0)
    fp = K.sum((1 - y_true) * y_pred, axis=0)
    fn = K.sum(y_true * (1 - y_pred), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = (1 + beta ** 2) * p * r / ((beta ** 2) * p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)

    return K.mean(f1)

In [0]:
def WaveNetResidualConv1D(num_filters, kernel_size, stacked_layer):

    def build_residual_block(l_input):
        resid_input = l_input
        for dilation_rate in [2**i for i in range(stacked_layer)]:
            l_sigmoid_conv1d = Conv1D(
              num_filters, kernel_size, dilation_rate=dilation_rate,
              padding='same', activation='sigmoid')(l_input)
            l_tanh_conv1d = Conv1D(
             num_filters, kernel_size, dilation_rate=dilation_rate,
             padding='same', activation='mish')(l_input)
            l_input = Multiply()([l_sigmoid_conv1d, l_tanh_conv1d])
            l_input = Conv1D(num_filters, 1, padding='same')(l_input)
            resid_input = Add()([resid_input ,l_input])
        return resid_input
    return build_residual_block


In [0]:
def WaveNetResidualConvBn1D(num_filters, kernel_size, stacked_layer):

    def build_bn_residual_block(l_input):
        resid_input = l_input
        for dilation_rate in [2**i for i in range(stacked_layer)]:
            conv1d1 = Conv1D(
              num_filters, kernel_size, dilation_rate=dilation_rate,
              padding='same')(l_input)
            bn_conv1d1 = BatchNormalization()(conv1d1)
            l_sigmoid_conv1d = Activation('sigmoid')(bn_conv1d1)
            conv1d2 = Conv1D(
              num_filters, kernel_size, dilation_rate=dilation_rate,
              padding='same')(l_input)
            bn_conv1d2 = BatchNormalization()(conv1d2)
            l_tanh_conv1d = Activation('mish')(bn_conv1d2)
            l_input = Multiply()([l_sigmoid_conv1d, l_tanh_conv1d])
            l_input = Conv1D(num_filters, 1, padding='same')(l_input)
            resid_input = Add()([resid_input ,l_input])
        return resid_input
    return build_bn_residual_block

In [0]:
def Classifier(shape_):
    num_filters_ = 16
    kernel_size_ = 3
    stacked_layers_ = [12, 8, 4, 1]
    l_input = Input(shape=(shape_))
    x = Conv1D(num_filters_, 1, padding='same')(l_input)
    x = WaveNetResidualConv1D(num_filters_, kernel_size_, stacked_layers_[0])(x)
    x = Conv1D(num_filters_*2, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*2, kernel_size_, stacked_layers_[1])(x)
    x = Conv1D(num_filters_*4, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*4, kernel_size_, stacked_layers_[2])(x)
    x = Conv1D(num_filters_*8, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*8, kernel_size_, stacked_layers_[3])(x)
    l_output = Dense(11, activation='softmax')(x)
    model = models.Model(inputs=[l_input], outputs=[l_output])
    opt = Adam(lr=LR)
    opt = tfa.optimizers.SWA(opt)
#      MacroF1(model, valid_x,valid_y)
    model.compile(loss=losses.CategoricalCrossentropy(), optimizer=opt, 
                  metrics=['accuracy', Precision(), Recall(), get_f1])
    return model

In [0]:
# def weighted_categorical_crossentropy(weights):
#     """
#     A weighted version of keras.objectives.categorical_crossentropy
    
#     Variables:
#         weights: numpy array of shape (C,) where C is the number of classes
    
#     Usage:
#         weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
#         loss = weighted_categorical_crossentropy(weights)
#         model.compile(loss=loss,optimizer='adam')
#     """
    
#     weights = K.variable(weights)
        
#     def loss(y_true, y_pred):
#         # scale predictions so that the class probas of each sample sum to 1
#         y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
#         # clip to prevent NaN's and Inf's
#         y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
#         # calc
#         loss = y_true * K.log(y_pred) * weights
#         loss = -K.sum(loss, -1)
#         return loss
    
#     return loss

In [0]:
# def w_categorical_crossentropy(weights):
#     """
#     Keras-style categorical crossentropy loss function, with weighting for each class.

#     Parameters
#     ----------
#     y_true : Tensor
#         Truth labels.
#     y_pred : Tensor
#         Predicted values.
#     weights: Tensor
#         Multiplicative factor for loss per class.

#     Returns
#     -------
#     loss : Tensor
#         Weighted crossentropy loss between labels and predictions.
#     """
#     weights = K.variable(weights)
#     def loss(y_true, y_pred):
#         y_true_max = K.argmax(y_true, axis=-1)
#         weighted_true = K.gather(weights, y_true_max)
#         loss = K.categorical_crossentropy(y_pred, y_true)*weighted_true
#         return loss

#     return loss 

In [0]:
# # # Custom loss function with costs
# def w_categorical_crossentropy(y_true, y_pred, weights):
#     nb_cl = len(weights)
#     final_mask = K.zeros_like(y_pred[:, 0])
#     y_pred_max = K.max(y_pred, axis=1)
#     y_pred_max = K.expand_dims(y_pred_max, 1)
#     y_pred_max_mat = K.equal(y_pred, y_pred_max)
#     for c_p, c_t in product(range(nb_cl), range(nb_cl)):
#         final_mask += (K.cast(weights[c_t, c_p],K.floatx()) * K.cast(y_pred_max_mat[:, c_p] ,K.floatx())* 
#                        K.cast(y_true[:, c_t],K.floatx()))
#     return K.categorical_crossentropy(y_pred, y_true) * final_mask

# w_array = np.ones((3,3))
# w_array[2,1] = 1.2
# w_array[1,2] = 1.2
# ncce = partial(w_categorical_crossentropy, weights=w_array)
# ncce.__name__ ='w_categorical_crossentropy'

In [0]:
def ClassifierW(shape_, weights_):
    num_filters_ = 16
    kernel_size_ = 3
    stacked_layers_ = [12, 8, 4, 1]*2
    l_input = Input(shape=(shape_))
    x = Conv1D(num_filters_, 1, padding='same')(l_input)
    x = WaveNetResidualConv1D(num_filters_, kernel_size_, stacked_layers_[0])(x)
    x = Conv1D(num_filters_*2, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*2, kernel_size_, stacked_layers_[1])(x)
    x = Conv1D(num_filters_*4, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*4, kernel_size_, stacked_layers_[2])(x)
    x = Conv1D(num_filters_*8, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*8, kernel_size_, stacked_layers_[3])(x)
    l_output = Dense(11, activation='softmax')(x)
    model = models.Model(inputs=[l_input], outputs=[l_output])
    opt = Adam(lr=LR)
    opt = tfa.optimizers.SWA(opt)

#      MacroF1(model, valid_x,valid_y)
    model.compile(loss = [multi_category_focal_loss2(alpha=0.25, gamma=2)], optimizer=opt, 
                  metrics=['accuracy', Precision(), Recall(), get_f1])

    # model.compile(loss = [categorical_focal_loss(alpha=0.25, gamma=2)], optimizer=opt, 
    #               metrics=['accuracy', Precision(), Recall(), get_f1])    

#     model.compile(loss = [cb_focal_loss(weights_, alpha=0.25, gamma=2)], optimizer=opt, 
#                   metrics=['accuracy', Precision(), Recall(), get_f1])    
    return model

In [0]:
# # if tf.keras.__version__ > "2.2.0":
# #     from tensorflow.keras.backend import normalize_data_format
# # else:
# #     from tensorflow.keras.utils.conv_utils import normalize_data_format

# class DropBlock1D(tf.keras.layers.Layer):
#     """See: https://arxiv.org/pdf/1810.12890.pdf"""

#     def __init__(self,
#                  block_size,
#                  keep_prob,
#                  sync_channels=False,
#                  data_format=None,
#                  **kwargs):
#         """Initialize the layer.
#         :param block_size: Size for each mask block.
#         :param keep_prob: Probability of keeping the original feature.
#         :param sync_channels: Whether to use the same dropout for all channels.
#         :param data_format: 'channels_first' or 'channels_last' (default).
#         :param kwargs: Arguments for parent class.
#         """
#         super(DropBlock1D, self).__init__(**kwargs)
#         self.block_size = block_size
#         self.keep_prob = keep_prob
#         self.sync_channels = sync_channels
#         self.data_format = 'channels_last' #K.image_data_format()
#         self.input_spec = tf.keras.layers.InputSpec(ndim=3)
#         self.supports_masking = True

#     def get_config(self):
#         config = {'block_size': self.block_size,
#                   'keep_prob': self.keep_prob,
#                   'sync_channels': self.sync_channels,
#                   'data_format': self.data_format}
#         # print("config:", config)
#         base_config = super(DropBlock1D, self).get_config()
#         return dict(list(base_config.items()) + list(config.items()))

#     def compute_mask(self, inputs, mask=None):
#         return mask

#     def compute_output_shape(self, input_shape):
#         return input_shape

#     def _get_gamma(self, feature_dim):
#         """Get the number of activation units to drop"""
#         feature_dim = K.cast(feature_dim, K.floatx())
#         # print("feature_dim:", K.get_value(feature_dim))
#         block_size = K.constant(self.block_size, dtype=K.floatx())
#         gamma = ((1.0 - self.keep_prob) / block_size) * (feature_dim / (feature_dim - block_size + 1.0))
#         print("gamma:", K.print_tensor(gamma))
#         return gamma

#     # def _compute_valid_seed_region(self, seq_length):
#     #     positions = K.arange(start = 0, stop = seq_length)
#     #     half_block_size = self.block_size // 2
#     #     valid_seed_region = K.switch(
#     #         K.all(
#     #             K.stack(
#     #                 [
#     #                     positions >= half_block_size,
#     #                     positions < seq_length - half_block_size,
#     #                 ],
#     #                 axis=-1,
#     #             ),
#     #             axis=-1,
#     #         ),
#     #         K.ones((seq_length,)),
#     #         K.zeros((seq_length,)),
#     #     )
#     #     return K.expand_dims(K.expand_dims(valid_seed_region, axis=0), axis=-1)

#     def _compute_drop_mask(self, shape):
#         print("shape:", K.print_tensor(shape))
#         seq_length = shape[1]
#         print("seq_length:", K.print_tensor(seq_length))
#         mask = K.random_binomial(shape, p=self._get_gamma(seq_length))

#         positions = K.arange(start = 0, stop = seq_length)
#         half_block_size = self.block_size // 2
#         valid_seed_region = K.switch(
#             K.all(
#                 K.stack(
#                     [
#                         positions >= half_block_size,
#                         positions < seq_length - half_block_size,
#                     ],
#                     axis=-1,
#                 ),
#                 axis=-1,
#             ),
#             K.ones((seq_length,)),
#             K.zeros((seq_length,)),
#         )
#         valid_seed_region_ex = K.expand_dims(K.expand_dims(valid_seed_region, axis=0), axis=-1)
#         mask *= valid_seed_region_ex
#         # mask *= self._compute_valid_seed_region(seq_length)
#         mask = MaxPool1D(
#             pool_size=self.block_size,
#             padding='same',
#             strides=1,
#             data_format='channels_last',
#         )(mask)
#         return 1.0 - mask

#     def call(self, inputs, training=None):

#         def dropped_inputs():
#             outputs = inputs
#             if self.data_format == 'channels_first':
#                 outputs = K.permute_dimensions(outputs, [0, 2, 1])
#             shape = K.shape(outputs)
#             if self.sync_channels:
#                 mask = self._compute_drop_mask([shape[0], shape[1], 1])
#             else:
#                 mask = self._compute_drop_mask(shape)
#             outputs = outputs * mask *\
#                 (K.cast(K.prod(shape), dtype=K.floatx()) / K.sum(mask))
#             if self.data_format == 'channels_first':
#                 outputs = K.permute_dimensions(outputs, [0, 2, 1])
#             return outputs

#         return K.in_train_phase(dropped_inputs, inputs, training=training)

In [0]:
def ClassifierG(shape_, weights_):
    num_filters_ = 16
    kernel_size_ = 3
    stacked_layers_ = [12, 8, 4, 1]*2
    drop_block_size = [64, 16, 4]
    l_input = Input(shape=(shape_))
    x = Conv1D(num_filters_, 1, padding='same')(l_input)
    x = WaveNetResidualConv1D(num_filters_, kernel_size_, stacked_layers_[0])(x)
    # x = DropBlock1D(block_size=drop_block_size[0], keep_prob=0.8)(x)
    x = BatchNormalization()(x)
    x = Conv1D(num_filters_*2, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*2, kernel_size_, stacked_layers_[1])(x)
    x = BatchNormalization()(x)
    # x = DropBlock1D(block_size=drop_block_size[1], keep_prob=0.8)(x)
    x = Conv1D(num_filters_*4, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*4, kernel_size_, stacked_layers_[2])(x)
    x = BatchNormalization()(x)
    x = Conv1D(num_filters_*8, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*8, kernel_size_, stacked_layers_[3])(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    # x = DropBlock1D(block_size=drop_block_size[2], keep_prob=0.8)(x)
    x = TimeDistributed(Dense(64, activation='mish'))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    l_output = TimeDistributed(Dense(11, activation='softmax'))(x)
    model = models.Model(inputs=[l_input], outputs=[l_output])
    opt = Adam(lr=LR)
    opt = tfa.optimizers.SWA(opt)

#      MacroF1(model, valid_x,valid_y)
    model.compile(loss = [multi_category_focal_loss2(alpha=0.25, gamma=2)], optimizer=opt, 
                  metrics=['accuracy', Precision(), Recall(), get_f1])

    # model.compile(loss = [categorical_focal_loss(alpha=0.25, gamma=2)], optimizer=opt, 
    #               metrics=['accuracy', Precision(), Recall(), get_f1])    

#     model.compile(loss = [cb_focal_loss(weights_, alpha=0.25, gamma=2)], optimizer=opt, 
#                   metrics=['accuracy', Precision(), Recall(), get_f1])    
    return model

In [0]:
def ClassifierD(shape_, weights_):
    num_filters_ = 16
    kernel_size_ = 3
    stacked_layers_ = [12, 8, 4, 1]*2
    drop_block_size = [64, 16, 4]
    l_input = Input(shape=(shape_))
    x = Conv1D(num_filters_, 1, padding='same')(l_input)
    x = WaveNetResidualConv1D(num_filters_, kernel_size_, stacked_layers_[0])(x)
    # x = SpatialDropout1D(0.3)(x)
    x = Conv1D(num_filters_*2, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*2, kernel_size_, stacked_layers_[1])(x)
    # x = SpatialDropout1D(0.3)(x)
    x = Conv1D(num_filters_*4, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*4, kernel_size_, stacked_layers_[2])(x)
    # x = SpatialDropout1D(0.2)(x)
    x = Conv1D(num_filters_*8, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*8, kernel_size_, stacked_layers_[3])(x)
    x = SpatialDropout1D(0.2)(x)
    x = TimeDistributed(Dense(64, activation='mish'))(x)
    x = SpatialDropout1D(0.3)(x)
    x = TimeDistributed(Dense(64, activation='mish'))(x)
    x = SpatialDropout1D(0.3)(x)
    l_output = TimeDistributed(Dense(11, activation='softmax'))(x)
    model = models.Model(inputs=[l_input], outputs=[l_output])
    opt = Adam(lr=LR)
    opt = tfa.optimizers.SWA(opt)

#      MacroF1(model, valid_x,valid_y)
    model.compile(loss = [multi_category_focal_loss2(alpha=0.25, gamma=2)], optimizer=opt, 
                  metrics=['accuracy', Precision(), Recall(), get_f1])

    # model.compile(loss = [categorical_focal_loss(alpha=0.25, gamma=2)], optimizer=opt, 
    #               metrics=['accuracy', Precision(), Recall(), get_f1])    

#     model.compile(loss = [cb_focal_loss(weights_, alpha=0.25, gamma=2)], optimizer=opt, 
#                   metrics=['accuracy', Precision(), Recall(), get_f1])    
    return model

In [0]:
class LayerNormalization(Layer):
	def __init__(self, eps=1e-6, **kwargs):
		self.eps = eps
		super(LayerNormalization, self).__init__(**kwargs)
	def build(self, input_shape):
		self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:], initializer=Ones(), trainable=True)
		self.beta = self.add_weight(name='beta', shape=input_shape[-1:], initializer=Zeros(), trainable=True)
		super(LayerNormalization, self).build(input_shape)
	def call(self, x):
		mean = K.mean(x, axis=-1, keepdims=True)
		std = K.std(x, axis=-1, keepdims=True)
		return self.gamma * (x - mean) / (std + self.eps) + self.beta
	def compute_output_shape(self, input_shape):
		return input_shape
	def get_config(self):
		config = super().get_config().copy()
		config.update({
				'eps': self.eps
		})
		return config

In [0]:
class PositionwiseFeedForward():
	def __init__(self, d_hid, d_inner_hid, dropout=0.1):
		self.w_1 = Conv1D(d_inner_hid, 1, activation='relu')
		self.w_2 = Conv1D(d_hid, 1)
		self.layer_norm = LayerNormalization()
		self.dropout = Dropout(dropout)
	def __call__(self, x):
		output = self.w_1(x) 
		output = self.w_2(output)
		output = self.dropout(output)
		output = Add()([output, x])
		return self.layer_norm(output)
	def get_config(self):
		config = super().get_config().copy()
		return config

In [0]:
class LRSchedulerPerStep(Callback):
    def __init__(self, d_model, warmup=4000):
        self.basic = d_model**-0.5
        self.warm = warmup**-1.5
        self.step_num = 0
    def on_batch_begin(self, batch, logs = None):
        self.step_num += 1
        lr = self.basic * min(self.step_num**-0.5, self.step_num*self.warm)
        K.set_value(self.model.optimizer.lr, lr)
    def on_epoch_begin(self, epoch, logs = None):
        print("learning rate: ", self.model.optimizer.lr)

In [0]:
def ClassifierD2(shape_, weights_):
    num_filters_ = 16
    kernel_size_ = 3
    stacked_layers_ = [12, 8, 4, 1]*2
    d_inner_hid = 64
    l_input = Input(shape=(shape_))
    x = Conv1D(num_filters_, 1, padding='same')(l_input)
    x = WaveNetResidualConv1D(num_filters_, kernel_size_, stacked_layers_[0])(x)
    x = LayerNormalization()(x)
    x = PositionwiseFeedForward(num_filters_, d_inner_hid)(x)

    x = Conv1D(num_filters_*2, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*2, kernel_size_, stacked_layers_[1])(x)
    x = LayerNormalization()(x)
    x = PositionwiseFeedForward(num_filters_*2, d_inner_hid*2)(x)

    x = Conv1D(num_filters_*4, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*4, kernel_size_, stacked_layers_[2])(x)
    x = LayerNormalization()(x)
    x = PositionwiseFeedForward(num_filters_*4, d_inner_hid*2)(x)

    x = Conv1D(num_filters_*8, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*8, kernel_size_, stacked_layers_[3])(x)
    x = LayerNormalization()(x)
    x = PositionwiseFeedForward(num_filters_*8, d_inner_hid*4)(x)
    
    x = SpatialDropout1D(0.2)(x)
    x = TimeDistributed(Dense(64, activation='mish'))(x)
    x = SpatialDropout1D(0.3)(x)
    x = TimeDistributed(Dense(64, activation='mish'))(x)
    x = SpatialDropout1D(0.3)(x)
    l_output = TimeDistributed(Dense(11, activation='softmax'))(x)
    model = models.Model(inputs=[l_input], outputs=[l_output])
    opt = Adam(lr=LR)
    opt = tfa.optimizers.SWA(opt)

#      MacroF1(model, valid_x,valid_y)
    model.compile(loss = [multi_category_focal_loss2(alpha=0.25, gamma=2)], optimizer=opt, 
                  metrics=['accuracy', Precision(), Recall(), get_f1])

    # model.compile(loss = [categorical_focal_loss(alpha=0.25, gamma=2)], optimizer=opt, 
    #               metrics=['accuracy', Precision(), Recall(), get_f1])    

#     model.compile(loss = [cb_focal_loss(weights_, alpha=0.25, gamma=2)], optimizer=opt, 
#                   metrics=['accuracy', Precision(), Recall(), get_f1])    
    return model

In [0]:
def ClassifierD2(shape_, weights_):
    num_filters_ = 16
    kernel_size_ = 3
    stacked_layers_ = [12, 8, 4, 1]*2
    d_inner_hid = 64
    l_input = Input(shape=(shape_))
    x = Conv1D(num_filters_, 1, padding='same')(l_input)
    x = WaveNetResidualConv1D(num_filters_, kernel_size_, stacked_layers_[0])(x)
    x = LayerNormalization()(x)
    x = PositionwiseFeedForward(num_filters_, d_inner_hid)(x)

    x = Conv1D(num_filters_*2, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*2, kernel_size_, stacked_layers_[1])(x)
    x = LayerNormalization()(x)
    x = PositionwiseFeedForward(num_filters_*2, d_inner_hid*2)(x)

    x = Conv1D(num_filters_*4, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*4, kernel_size_, stacked_layers_[2])(x)
    x = LayerNormalization()(x)
    x = PositionwiseFeedForward(num_filters_*4, d_inner_hid*2)(x)

    x = Conv1D(num_filters_*8, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*8, kernel_size_, stacked_layers_[3])(x)
    x = LayerNormalization()(x)
    x = PositionwiseFeedForward(num_filters_*8, d_inner_hid*4)(x)
    
    x = SpatialDropout1D(0.2)(x)
    x = TimeDistributed(Dense(64, activation='mish'))(x)
    x = SpatialDropout1D(0.3)(x)
    x = TimeDistributed(Dense(64, activation='mish'))(x)
    x = SpatialDropout1D(0.3)(x)
    l_output = TimeDistributed(Dense(11, activation='softmax'))(x)
    model = models.Model(inputs=[l_input], outputs=[l_output])
    opt = Adam(lr=LR)
    opt = tfa.optimizers.SWA(opt)

#      MacroF1(model, valid_x,valid_y)
    model.compile(loss = [multi_category_focal_loss2(alpha=0.25, gamma=2)], optimizer=opt, 
                  metrics=['accuracy', Precision(), Recall(), get_f1])

    # model.compile(loss = [categorical_focal_loss(alpha=0.25, gamma=2)], optimizer=opt, 
    #               metrics=['accuracy', Precision(), Recall(), get_f1])    

#     model.compile(loss = [cb_focal_loss(weights_, alpha=0.25, gamma=2)], optimizer=opt, 
#                   metrics=['accuracy', Precision(), Recall(), get_f1])    
    return model

In [0]:
def Classifierx(shape_):        
    #dsize = [256, 512, 256, 128, 11]
    dsize = [64, 128, 64, 32, 11]  
    inp = Input(shape=(shape_))
    x = Bidirectional(GRU(dsize[0], return_sequences=True))(inp)
    x = Attention(dsize[1])(x)
    x = TimeDistributed(Dense(dsize[2], activation='mish'))(x)
    x = TimeDistributed(Dense(dsize[3], activation='mish'))(x)
    out = TimeDistributed(Dense(dsize[4], activation='softmax', name='out'))(x)
    
    model = models.Model(inputs=inp, outputs=out) 
    
    opt = Adam(lr=LR)
    opt = tfa.optimizers.SWA(opt)
    model.compile(loss=losses.CategoricalCrossentropy(), optimizer=opt, 
                  metrics=['accuracy'])
    return model

In [0]:

class MacroF1ES(Callback):
    def __init__(self, model, inputs, targets, 
                 patience=5, delta=0, checkpoint_path='checkpoint.h5', is_maximize=True):
        
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis=2).reshape(-1)
        self.patience, self.delta, self.checkpoint_path = patience, delta, checkpoint_path
        self.counter, self.best_score = 0, None
        self.is_maximize = is_maximize
        self.stopped_epoch = 0
        
    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
        score = f1_score(self.targets, pred, average="macro")
        logger.info(f'\n epoch:{epoch:03d}, F1Macro: {score:.5f}')   
        
        if self.best_score is None or \
        (score > self.best_score + self.delta if self.is_maximize else score < self.best_score - self.delta):
            self.model.save_weights(self.checkpoint_path)
#             torch.save(model.state_dict(), self.checkpoint_path)
            self.best_score, self.counter = score, 0
        else:
            self.counter += 1
            if self.counter >= self.patience: ##stop training
                self.stopped_epoch = epoch
                self.model.stop_training = True
                
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
              logger.info('Epoch %05d: early stopping' % (self.stopped_epoch + 1))   

In [0]:
class MacroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis=2).reshape(-1)

    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
        score = f1_score(self.targets, pred, average="macro")
        print(f' F1Macro: {score:.5f}')    

In [0]:
def normalize(train, test):
    
    train_input_mean = train.signal.mean()
    train_input_sigma = train.signal.std()
    train['signal'] = (train.signal-train_input_mean)/train_input_sigma
    test['signal'] = (test.signal-train_input_mean)/train_input_sigma

    return train, test

In [0]:

def run_everything(fe_config : List) -> NoReturn:
    not_feats_cols = ['time']
    target_col = ['open_channels']
    init_logger()
    with timer(f'Reading Data'):
        logger.info('Reading Data Started ...')
        base = os.path.abspath(PATH+'liverpool-ion-switching/')
        train, test, sample_submission = read_data(base)
        train, test = normalize(train, test)    
        logger.info('Reading and Normalizing Data Completed ...')
    with timer(f'Creating Features'):
        logger.info('Feature Enginnering Started ...')
        for config in fe_config:
            train = run_feat_enginnering(train, create_all_data_feats=config[0], batch_size=config[1])
            train = reduce_mem_usage(train)
            test  = run_feat_enginnering(test,  create_all_data_feats=config[0], batch_size=config[1])
            test = reduce_mem_usage(test)
        train, test, feats = feature_selection(train, test)
        logger.info('Feature Enginnering Completed ...')

    with timer(f'Running Wavenet model'):
        logger.info(f'Training Wavenet model with {SPLITS} folds of GroupKFold Started ...')
        run_cv_model_by_batch(train, test, splits=SPLITS, batch_col='group', feats=feats, 
                              sample_submission=sample_submission, nn_epochs=EPOCHS, nn_batch_size=NNBATCHSIZE)
        logger.info(f'Training completed ...')


In [0]:
run_everything(fe_config)

2020-04-20 07:07:40,054 INFO Reading Data Started ...
2020-04-20 07:07:42,464 INFO Reading and Normalizing Data Completed ...
2020-04-20 07:07:42,465 INFO [Reading Data] done in 2 s
2020-04-20 07:07:42,468 INFO Feature Enginnering Started ...


Mem. usage decreased to 171.66 MB (70.5 % reduction)
Mem. usage decreased to 61.04 MB (72.9 % reduction)
['signal', 'signal_shift_pos_1', 'signal_shift_neg_1', 'signal_shift_pos_2', 'signal_shift_neg_2', 'signal_shift_pos_3', 'signal_shift_neg_3', 'signal_shift_pos_1_msignal', 'signal_shift_neg_1_msignal', 'signal_shift_pos_2_msignal', 'signal_shift_neg_2_msignal', 'signal_shift_pos_3_msignal', 'signal_shift_neg_3_msignal', 'signal_2']


2020-04-20 07:07:51,294 INFO Feature Enginnering Completed ...
2020-04-20 07:07:51,299 INFO [Creating Features] done in 9 s
2020-04-20 07:07:51,302 INFO Training Wavenet model with 5 folds of GroupKFold Started ...


ion_switch/res/wavenet-lr-res/WaveNet-NormForward-LR-FOCALLOSS_len4000_lr0.001/20-Apr-2020-07
weights_: [ 0.36652399  0.46106257  0.82059173  0.67983748  1.12675802  1.63577934
  2.41635544  1.71516878  1.85390282  3.33929955 12.72060713]
nums_: 0     1240152
1      985865
2      553924
3      668609
4      403410
5      277877
6      188112
7      265015
8      245183
9      136120
10      35733
Name: open_channels, dtype: int64
cb_weights_: [0.97174558 0.97174558 0.97174558 0.97174558 0.97174567 0.97176004
 0.97227024 0.97176977 0.97179906 0.97596    1.27771292]
model initilization done!


learning rate:  <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.001>
Epoch 1/300
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
100/100 [==============================] - ETA: 0s - loss: 0.0624 - accuracy: 0.1330 - precision: 0.2089 - recall: 0.0220 - get_f1: 0.0393

...

100/100 [==============================] - 21s 210ms/step - loss: 0.0232 - accuracy: 0.4206 - precision_1: 0.9172 - recall_1: 0.1523 - get_f1: 0.2546 - val_loss: 0.0225 - val_accuracy: 0.4188 - val_precision_1: 0.9189 - val_recall_1: 0.1558 - val_get_f1: 0.2098
learning rate:  <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.00085051724>
Epoch 55/300
100/100 [==============================] - ETA: 0s - loss: 0.0233 - accuracy: 0.4186 - precision_1: 0.9161 - recall_1: 0.1521 - get_f1: 0.2541

2020-04-20 08:07:06,704 INFO 
 epoch:054, F1Macro: 0.49715


100/100 [==============================] - 21s 210ms/step - loss: 0.0233 - accuracy: 0.4186 - precision_1: 0.9161 - recall_1: 0.1521 - get_f1: 0.2541 - val_loss: 0.0225 - val_accuracy: 0.4191 - val_precision_1: 0.9168 - val_recall_1: 0.1583 - val_get_f1: 0.2115
learning rate:  <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.00084274984>
Epoch 56/300
100/100 [==============================] - ETA: 0s - loss: 0.0233 - accuracy: 0.4138 - precision_1: 0.9171 - recall_1: 0.1528 - get_f1: 0.2554

2020-04-20 08:07:27,873 INFO 
 epoch:055, F1Macro: 0.49675


100/100 [==============================] - 21s 210ms/step - loss: 0.0233 - accuracy: 0.4138 - precision_1: 0.9171 - recall_1: 0.1528 - get_f1: 0.2554 - val_loss: 0.0225 - val_accuracy: 0.4191 - val_precision_1: 0.9211 - val_recall_1: 0.1567 - val_get_f1: 0.2107
learning rate:  <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.0008351914>
Epoch 57/300
 63/100 [=================>............] - ETA: 6s - loss: 0.0233 - accuracy: 0.4280 - precision_1: 0.9166 - recall_1: 0.1513 - get_f1: 0.2538